<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); padding: 20px; border-radius: 10px; margin-bottom: 20px;">
    <h1 style="color: white; margin: 0; font-size: 36px;">⚙️ Notebook 3: Modules</h1>
    <p style="color: rgba(255,255,255,0.9); margin-top: 10px; font-size: 18px;">Making Things Happen - Execution Strategies for LLMs</p>
</div>

<div style="display: flex; justify-content: space-between; margin-bottom: 20px;">
    <a href="02_signatures.ipynb" style="text-decoration: none; padding: 10px 20px; background: #f0f0f0; border-radius: 5px;">← Notebook 2</a>
    <span style="padding: 10px 20px; background: #fff8e1; border-radius: 5px;">🟡 Intermediate • 20 minutes</span>
    <a href="04_providers_adapters.ipynb" style="text-decoration: none; padding: 10px 20px; background: #f0f0f0; border-radius: 5px;">Notebook 4 →</a>
</div>

## 🎯 What You'll Learn

<div style="background: #f5f5f5; padding: 20px; border-radius: 10px; border-left: 4px solid #667eea;">
    <ul style="margin: 0; padding-left: 20px;">
        <li>✅ <strong>Understand modules</strong> as execution strategies for signatures</li>
        <li>✅ <strong>Master core modules</strong>: Predict, ChainOfThought, Retry, Refine</li>
        <li>✅ <strong>Handle errors gracefully</strong> with retry and fallback patterns</li>
        <li>✅ <strong>Improve outputs iteratively</strong> with refinement strategies</li>
        <li>✅ <strong>Compose modules</strong> for complex workflows</li>
        <li>✅ <strong>Optimize performance</strong> with the right module choices</li>
        <li>✅ <strong>Build robust pipelines</strong> that handle real-world complexity</li>
    </ul>
</div>

## 🔧 Setup

In [1]:
import asyncio
import time
import os
from typing import List, Dict, Optional

from logillm.core.predict import Predict, ChainOfThought
from logillm.core.retry import Retry
from logillm.core.refine import Refine
from logillm.core.signatures import Signature, InputField, OutputField
from logillm.providers import create_provider, register_provider

# Check API key
if not os.getenv("OPENAI_API_KEY"):
    print("⚠️ WARNING: OPENAI_API_KEY not set!")
    print("Set it with: export OPENAI_API_KEY=your_key")
else:
    print("✅ OpenAI API key found")

# Setup provider
try:
    provider = create_provider("openai", model="gpt-4.1-mini")
    register_provider(provider, set_default=True)
    print("✅ LogiLLM ready with modules loaded!")
except Exception as e:
    print(f"❌ Error setting up provider: {e}")
    print("Make sure you have logillm[openai] installed and API key set")

✅ OpenAI API key found
✅ LogiLLM ready with modules loaded!


## 💡 Important: Adapters for Type Safety

<div style="background: #fff3cd; padding: 15px; border-radius: 8px; border-left: 4px solid #ffc107; margin: 20px 0;">
    <h4 style="margin-top: 0;">⚠️ Using JSONAdapter for List Fields</h4>
    <p>When your signature has <strong>list fields</strong> (like <code>list[str]</code> or <code>list[float]</code>), use the <strong>JSONAdapter</strong> for proper type parsing:</p>
    <pre style="background: #f8f9fa; padding: 10px; border-radius: 4px;">
from logillm.core.adapters import JSONAdapter

# For signatures with list fields
module = Predict(MySignature, adapter=JSONAdapter())
    </pre>
    <p>The default ChatAdapter works great for simple types, but JSONAdapter ensures lists are properly parsed as Python lists, not strings.</p>
</div>

## 📊 Module Comparison

Let's see how different modules handle the same task:

In [2]:
# Define a common task
class MathProblem(Signature):
    """Solve a word problem."""
    problem: str = InputField(desc="Math word problem to solve")
    answer: float = OutputField(desc="Numerical answer")
    unit: str = OutputField(desc="Unit of measurement if applicable")

test_problem = """A train travels at 60 mph for 2.5 hours, 
then slows to 40 mph for another 1.5 hours. 
What is the total distance traveled?"""

# 1. Basic Predict
print("🔹 Using Predict (basic):")
basic = Predict(MathProblem)
result = await basic(problem=test_problem)

# LogiLLM guarantees all output fields exist
print(f"  Answer: {result.outputs['answer']} {result.outputs['unit']}")
print(f"  Fields returned: {list(result.outputs.keys())}")

# 2. ChainOfThought - adds reasoning
print("\n🔸 Using ChainOfThought (with reasoning):")
cot = ChainOfThought(MathProblem)
result = await cot(problem=test_problem)

# All fields are guaranteed to exist
print(f"  Answer: {result.outputs['answer']} {result.outputs['unit']}")
print(f"  Reasoning: {result.outputs['reasoning'][:200]}...")
print(f"  Fields returned: {list(result.outputs.keys())}")

print("\n💡 Notice: ChainOfThought automatically added a 'reasoning' field!")

🔹 Using Predict (basic):
  Answer: 195.0 miles
  Fields returned: ['answer', 'unit']

🔸 Using ChainOfThought (with reasoning):
  Answer: 210.0 miles
  Reasoning: Let's think step by step to solve this problem. The train first travels at 60 mph for 2.5 hours. The distance covered in this part is speed multiplied by time: 60 mph * 2.5 hours = 150 miles. Then, th...
  Fields returned: ['reasoning', 'answer', 'unit']

💡 Notice: ChainOfThought automatically added a 'reasoning' field!


In [ ]:
# Advanced Predict features
from logillm.core.adapters import JSONAdapter

class ContentGeneration(Signature):
    """Generate content with specific requirements."""
    topic: str = InputField()
    tone: str = InputField(desc="writing tone: formal, casual, humorous")
    length: str = InputField(desc="short, medium, or long")
    
    title: str = OutputField(desc="Catchy title")
    content: str = OutputField(desc="Main content")
    tags: list[str] = OutputField(desc="3-5 relevant tags")

# Create predictor with JSONAdapter for proper list parsing
generator = Predict(
    ContentGeneration,
    adapter=JSONAdapter(),  # Use JSON adapter for proper list parsing
    config={
        'temperature': 0.8,  # More creative
        'max_tokens': 500
    }
)

result = generator.call_sync(
    topic="AI in healthcare",
    tone="casual",
    length="short"
)

print(f"📝 {result.outputs['title']}")
print(f"\n{result.outputs['content'][:200]}...")
print(f"\n🏷️ Tags: {', '.join(result.outputs['tags'])}")

## 🧠 Module 2: ChainOfThought - Reasoning Power

**ChainOfThought** automatically adds step-by-step reasoning to any signature, improving accuracy on complex tasks.

In [4]:
# Complex reasoning task
class LogicalPuzzle(Signature):
    """Solve a logical puzzle."""
    puzzle: str = InputField(desc="The puzzle to solve")
    solution: str = OutputField(desc="The answer to the puzzle")
    confidence: float = OutputField(desc="Confidence in solution (0-1)")

puzzle_text = """Three friends - Alice, Bob, and Charlie - are wearing red, blue, and green shirts.
Alice is not wearing red. The person in blue is standing between the other two.
Charlie is standing to the right of the person in green.
What color is each person wearing?"""

# Compare Predict vs ChainOfThought
print("🔴 Without reasoning (Predict):")
basic = Predict(LogicalPuzzle)
start = time.time()
result1 = await basic(puzzle=puzzle_text)
time1 = time.time() - start

# Safely get outputs
solution1 = result1.outputs.get('solution', 'N/A')
print(f"  Solution: {solution1}")

# Handle confidence as either float, string, or None
conf1 = result1.outputs.get('confidence', 0.5)
if conf1 is None:
    conf1 = 0.5
elif isinstance(conf1, str):
    try:
        conf1 = float(conf1)
    except:
        conf1 = 0.5
print(f"  Confidence: {conf1:.2f}")
print(f"  Time: {time1:.2f}s")

print("\n🟢 With reasoning (ChainOfThought):")
cot = ChainOfThought(LogicalPuzzle)
start = time.time()
result2 = await cot(puzzle=puzzle_text)
time2 = time.time() - start

# Safely get outputs
solution2 = result2.outputs.get('solution', 'N/A')
print(f"  Solution: {solution2}")

# Handle confidence safely
conf2 = result2.outputs.get('confidence', 0.5)
if conf2 is None:
    conf2 = 0.5
elif isinstance(conf2, str):
    try:
        conf2 = float(conf2)
    except:
        conf2 = 0.5
print(f"  Confidence: {conf2:.2f}")
print(f"  Time: {time2:.2f}s")

# Check for reasoning
reasoning = result2.outputs.get('reasoning')
if reasoning:
    print("\n  Reasoning process:")
    print(f"  {reasoning}")

# Safe division for performance comparison
if time1 > 0 and time2 > 0:
    print(f"\n📊 ChainOfThought is {time2/time1:.1f}x slower but often more accurate!")
else:
    print("\n📊 Performance comparison not available")

🔴 Without reasoning (Predict):
  Solution: - Alice is wearing green,
  - Bob is wearing red,
  - Charlie is wearing blue.
  Confidence: 0.50
  Time: 2.81s

🟢 With reasoning (ChainOfThought):
  Solution: Alice is wearing blue.

Bob is wearing green.

Charlie is wearing red.
  Confidence: 0.85
  Time: 9.22s

  Reasoning process:
  Let's think step by step to solve this problem.

We have three friends: Alice, Bob, and Charlie.

They are each wearing one of three colors: red, blue, or green.

Clues:

1. Alice is not wearing red.

2. The person in blue is standing between the other two.

3. Charlie is standing to the right of the person in green.

First, since the blue-shirted person is standing between the other two, their order must be:

Person A - Blue - Person B

So, the blue shirt wearer is in the middle position.

Positions (left to right):

Position 1 - Position 2 (blue) - Position 3

Charlie is standing to the right of the person in green, so:

Green must be to the left of Charlie.


In [5]:
# Complex reasoning task (duplicate cell - simplified)
class LogicalPuzzle(Signature):
    """Solve a logical puzzle."""
    puzzle: str = InputField(desc="The puzzle to solve")
    solution: str = OutputField(desc="The answer to the puzzle")
    confidence: float = OutputField(desc="Confidence in solution (0-1)")

puzzle_text = """Three friends - Alice, Bob, and Charlie - are wearing red, blue, and green shirts.
Alice is not wearing red. The person in blue is standing between the other two.
Charlie is standing to the right of the person in green.
What color is each person wearing?"""

# Compare Predict vs ChainOfThought
print("🔴 Without reasoning (Predict):")
basic = Predict(LogicalPuzzle)
start = time.time()
result1 = await basic(puzzle=puzzle_text)  # Keep using await - it works in Jupyter
time1 = time.time() - start

# Safely get solution
solution1 = result1.outputs.get('solution', 'N/A')
print(f"  Solution: {solution1}")

# Handle confidence as either float or string
conf1 = result1.outputs.get('confidence', 0.5)
if isinstance(conf1, str):
    try:
        conf1 = float(conf1)
    except:
        conf1 = 0.5
print(f"  Confidence: {conf1:.2f}")
print(f"  Time: {time1:.2f}s")

print("\n🟢 With reasoning (ChainOfThought):")
cot = ChainOfThought(LogicalPuzzle)
start = time.time()
result2 = await cot(puzzle=puzzle_text)  # Keep using await
time2 = time.time() - start

# Safely get solution
solution2 = result2.outputs.get('solution', 'N/A')
print(f"  Solution: {solution2}")

# Handle confidence safely
conf2 = result2.outputs.get('confidence', 0.5)
if isinstance(conf2, str):
    try:
        conf2 = float(conf2)
    except:
        conf2 = 0.5
print(f"  Confidence: {conf2:.2f}")
print(f"  Time: {time2:.2f}s")

# Check for reasoning field
reasoning = result2.outputs.get('reasoning')
if reasoning:
    print("\n  Reasoning process:")
    print(f"  {reasoning}")

# Calculate speedup only if both times are valid
if time1 > 0 and time2 > 0:
    print(f"\n📊 ChainOfThought is {time2/time1:.1f}x slower but often more accurate!")
else:
    print("\n📊 Performance comparison not available")

🔴 Without reasoning (Predict):
  Solution: N/A
  Confidence: 0.50
  Time: 6.10s

🟢 With reasoning (ChainOfThought):
  Solution: N/A
  Confidence: 0.50
  Time: 9.07s

  Reasoning process:
  Let's analyze the puzzle step by step.

📊 ChainOfThought is 1.5x slower but often more accurate!


In [ ]:
# Signature that might fail due to complexity
from logillm.core.adapters import JSONAdapter

class DataExtraction(Signature):
    """Extract structured data from messy text."""
    text: str = InputField(desc="Messy unstructured text")
    
    names: list[str] = OutputField(desc="All person names found")
    dates: list[str] = OutputField(desc="All dates in YYYY-MM-DD format")
    amounts: list[float] = OutputField(desc="All monetary amounts as floats")
    emails: list[str] = OutputField(desc="All email addresses")

# Wrap with Retry for resilience, use JSONAdapter for proper list parsing
robust_extractor = Retry(
    Predict(DataExtraction, adapter=JSONAdapter()),
    max_retries=3,
    base_delay=1.0,
    backoff_multiplier=2.0
)

messy_text = """Meeting notes 3/15/24: John Smith (jsmith@email.com) proposed 
$1,250.50 budget. Mary Johnson agreed. Follow up by march 20th 2024. 
Additional $500 approved. Contact: m.johnson@company.org by 2024-03-22."""

print("🔄 Extracting with retry protection (using JSONAdapter):")
result = robust_extractor.call_sync(text=messy_text)

print(f"\n📊 Extracted Data:")
print(f"  Names: {result.outputs['names']}")
print(f"  Dates: {result.outputs['dates']}")
print(f"  Amounts: ${', $'.join(str(amt) for amt in result.outputs['amounts'])}")
print(f"  Emails: {result.outputs['emails']}")

# Demonstrate retry on actual failure
class StrictValidation(Signature):
    """Generate data with strict requirements."""
    requirement: str = InputField()
    valid_json: dict = OutputField(desc="Must be valid JSON dict with 'id' and 'value' keys")

print("\n🚨 Testing retry on potential failures:")
strict_module = Retry(
    Predict(StrictValidation, adapter=JSONAdapter()),
    max_retries=3
)

try:
    result = strict_module.call_sync(
        requirement="Generate a config with exactly 5 items"
    )
    print(f"✅ Success: {result.outputs['valid_json']}")
except Exception as e:
    print(f"❌ Failed after retries: {e}")

## ✨ Module 4: Refine - Iterative Improvement

**Refine** takes an initial output and iteratively improves it through multiple passes.

In [ ]:
# Refine example with CORRECT API
class StoryGeneration(Signature):
    """Generate a creative story."""
    prompt: str = InputField(desc="Story prompt or theme")
    genre: str = InputField(desc="Story genre")
    
    story: str = OutputField(desc="The generated story")
    title: str = OutputField(desc="Story title")

# Define a reward function for story quality
def story_quality_reward(inputs: dict, prediction) -> float:
    """Evaluate story quality based on length and completion."""
    if not prediction.success or not prediction.outputs.get('story'):
        return 0.0
    
    story = prediction.outputs['story']
    title = prediction.outputs.get('title', '')
    
    # Reward based on: has title, reasonable length, complete sentences
    score = 0.0
    if title:
        score += 0.2
    if len(story) > 100:
        score += 0.3
    if len(story) > 200:
        score += 0.2
    if story.strip().endswith(('.', '!', '?', '"')):
        score += 0.3
    
    return min(score, 1.0)

# Create Refine with CORRECT parameters
story_refiner = Refine(
    module=Predict(StoryGeneration),  # The module to refine
    N=3,  # Try 3 times with different temperatures
    reward_fn=story_quality_reward,  # Evaluation function
    threshold=0.8,  # Stop when we reach 0.8 quality
    fail_count=2  # Allow up to 2 failures
)

print("📖 Story Generation with Refinement:\n")

# Generate with refinement
result = story_refiner.call_sync(  # Use call_sync() instead of await
    prompt="A robot discovers emotions for the first time",
    genre="sci-fi"
)

print(f"📝 REFINED STORY:")
print(f"Title: {result.outputs.get('title', 'Untitled')}")
print(f"\n{result.outputs.get('story', 'No story generated')[:400]}...")

# Check metadata for refinement details
if result.metadata and 'refinement_attempts' in result.metadata:
    print(f"\n📊 Refinement Stats:")
    print(f"  Attempts: {result.metadata['refinement_attempts']}")
    print(f"  Best reward: {result.metadata.get('best_reward', 0):.2f}")
    
print("\n💡 Refine uses temperature variation to explore different outputs!")

📖 Story Generation with Refinement:



## 🔗 Module Composition - Combining Strategies

Modules can be composed to create powerful pipelines:

In [ ]:
# Complex signature requiring multiple strategies
class TechnicalAnalysis(Signature):
    """Analyze technical documentation."""
    documentation: str = InputField(desc="Technical documentation text")
    
    summary: str = OutputField(desc="Executive summary")
    key_concepts: list[str] = OutputField(desc="Main technical concepts")
    complexity_score: int = OutputField(desc="Complexity from 1-10")
    prerequisites: list[str] = OutputField(desc="Required knowledge")

# Compose modules: ChainOfThought wrapped in Retry
robust_analyzer = Retry(
    ChainOfThought(TechnicalAnalysis),  # Add reasoning
    max_retries=3,
    base_delay=1.0,
    backoff_multiplier=2.0
)

# Test with technical content
tech_doc = """
Kubernetes uses a declarative API model where you describe the desired state 
of your application using YAML manifests. The control plane continuously 
reconciles the actual state with the desired state through controllers. 
Pods are the smallest deployable units, while Services provide stable 
networking endpoints. ConfigMaps and Secrets manage configuration.
"""

print("🔧 Analyzing with composed modules:\n")
result = await robust_analyzer(documentation=tech_doc)

# Check if the call succeeded
if not result.success:
    print(f"❌ Analysis failed: {result.error}")
    print("Make sure your API key is set and you have credits")
else:
    print(f"📝 Summary: {result.outputs['summary']}")
    
    # Note: Due to a current LogiLLM limitation, list fields might come back as strings
    # Here's a workaround to parse them:
    def parse_list_field(value):
        if isinstance(value, list):
            return value
        elif isinstance(value, str):
            # Try to parse comma-separated or bullet lists
            if ',' in value:
                return [item.strip() for item in value.split(',')]
            elif '\n' in value:
                return [line.strip('- •* ') for line in value.split('\n') if line.strip()]
            else:
                return [value]  # Single item
        return []
    
    key_concepts = parse_list_field(result.outputs['key_concepts'])
    print(f"\n🎯 Key Concepts: {', '.join(key_concepts)}")
    
    print(f"\n📊 Complexity: {result.outputs['complexity_score']}/10")
    
    prerequisites = parse_list_field(result.outputs['prerequisites'])
    print(f"\n📚 Prerequisites: {', '.join(prerequisites)}")
    
    # ChainOfThought adds reasoning automatically
    print(f"\n💭 Reasoning: {result.outputs['reasoning'][:200]}...")

## 🤖 Brief: ReAct & Tools (Advanced)

**ReAct** combines reasoning with actions (tool use) for agent-like behavior. We'll cover this in detail in Notebook 6.

In [ ]:
# Quick preview of ReAct pattern
from logillm.core.react import ReAct
from logillm.core.tools import Tool

# Define simple tools
def calculate(expression: str) -> float:
    """Evaluate a mathematical expression."""
    try:
        # Safe evaluation using ast.literal_eval for safety
        import ast
        import operator as op
        
        # Supported operators
        ops = {ast.Add: op.add, ast.Sub: op.sub, ast.Mult: op.mul,
               ast.Div: op.truediv, ast.Pow: op.pow, ast.USub: op.neg}
        
        def eval_expr(node):
            if isinstance(node, ast.Constant):
                return node.value
            elif isinstance(node, ast.BinOp):
                return ops[type(node.op)](eval_expr(node.left), eval_expr(node.right))
            elif isinstance(node, ast.UnaryOp):
                return ops[type(node.op)](eval_expr(node.operand))
            else:
                raise TypeError(f"Unsupported type {node}")
        
        tree = ast.parse(expression, mode='eval')
        return eval_expr(tree.body)
    except:
        return 0.0

def get_date() -> str:
    """Get current date."""
    from datetime import datetime
    return datetime.now().strftime("%Y-%m-%d")

# Create tools with CORRECT parameter name
calc_tool = Tool(
    name="calculator",
    func=calculate,
    desc="Calculate mathematical expressions"  # CORRECT: desc not description
)

date_tool = Tool(
    name="get_date",
    func=get_date,
    desc="Get today's date"  # CORRECT: desc not description
)

# ReAct agent with tools - CORRECT PARAMETER
class ProblemSolving(Signature):
    """Solve problems using available tools."""
    problem: str = InputField(desc="Problem to solve")
    answer: str = OutputField(desc="Final answer")
    
agent = ReAct(
    ProblemSolving,
    tools=[calc_tool, date_tool],
    max_iters=3  # CORRECT PARAMETER (was max_steps)
)

# Test the agent
result = agent.call_sync(  # Use call_sync() instead of await
    problem="If today is a weekday and I work 8 hours at $25/hour, how much do I earn?"
)

print("🤖 ReAct Agent Result:")
print(f"  Answer: {result.outputs['answer']}")
print("\n📝 Note: ReAct enables tool use for complex problem solving!")
print("    We'll explore this more in Notebook 6.")

## 📊 Module Selection Guide

<div style="background: #f8f9fa; padding: 20px; border-radius: 10px; margin: 20px 0;">
    <h3 style="margin-top: 0;">When to Use Each Module</h3>
    <table style="width: 100%; border-collapse: collapse;">
        <tr style="background: #e9ecef;">
            <th style="padding: 10px; text-align: left;">Module</th>
            <th style="padding: 10px; text-align: left;">Use When</th>
            <th style="padding: 10px; text-align: left;">Avoid When</th>
        </tr>
        <tr>
            <td style="padding: 10px; border-top: 1px solid #dee2e6;"><strong>Predict</strong></td>
            <td style="padding: 10px; border-top: 1px solid #dee2e6;">Simple transformations, speed matters</td>
            <td style="padding: 10px; border-top: 1px solid #dee2e6;">Complex reasoning needed</td>
        </tr>
        <tr>
            <td style="padding: 10px; border-top: 1px solid #dee2e6;"><strong>ChainOfThought</strong></td>
            <td style="padding: 10px; border-top: 1px solid #dee2e6;">Math, logic, multi-step problems</td>
            <td style="padding: 10px; border-top: 1px solid #dee2e6;">Simple lookups, speed critical</td>
        </tr>
        <tr>
            <td style="padding: 10px; border-top: 1px solid #dee2e6;"><strong>Retry</strong></td>
            <td style="padding: 10px; border-top: 1px solid #dee2e6;">Network issues possible, parsing complex data</td>
            <td style="padding: 10px; border-top: 1px solid #dee2e6;">Deterministic operations</td>
        </tr>
        <tr>
            <td style="padding: 10px; border-top: 1px solid #dee2e6;"><strong>Refine</strong></td>
            <td style="padding: 10px; border-top: 1px solid #dee2e6;">Quality matters more than speed</td>
            <td style="padding: 10px; border-top: 1px solid #dee2e6;">Real-time requirements</td>
        </tr>
        <tr>
            <td style="padding: 10px; border-top: 1px solid #dee2e6;"><strong>ReAct</strong></td>
            <td style="padding: 10px; border-top: 1px solid #dee2e6;">Need external tools/data</td>
            <td style="padding: 10px; border-top: 1px solid #dee2e6;">Simple transformations</td>
        </tr>
    </table>
</div>

## 🎮 Interactive Exercise: Build a Robust Pipeline

Create a complete pipeline combining multiple modules:

In [ ]:
# Exercise: Build a robust code review system
from logillm.core.adapters import JSONAdapter

class CodeReview(Signature):
    """Review code for quality and issues."""
    code: str = InputField(desc="Code to review")
    language: str = InputField(desc="Programming language")
    
    issues: list[str] = OutputField(desc="List of issues found")
    suggestions: list[str] = OutputField(desc="Improvement suggestions")
    score: int = OutputField(desc="Quality score 1-10")
    security_risks: list[str] = OutputField(desc="Security vulnerabilities")

# Create a robust pipeline that:
# 1. Uses ChainOfThought for better analysis
# 2. Uses JSONAdapter for proper list parsing
# 3. Wraps with Retry for reliability
robust_reviewer = Retry(
    ChainOfThought(CodeReview, adapter=JSONAdapter()),  # Use JSON adapter
    max_retries=3,
    base_delay=0.5
)

# Test with sample code
sample_code = """
def process_user_input(user_data):
    # Process user data
    query = "SELECT * FROM users WHERE id = " + user_data['id']
    result = database.execute(query)
    return result[0]['password']  # Return user password
"""

print("🔍 Code Review Pipeline Test (with JSONAdapter):\n")
result = await robust_reviewer(
    code=sample_code,
    language="python"
)

print(f"📊 Quality Score: {result.outputs['score']}/10")

# With JSONAdapter, lists should be properly parsed
issues = result.outputs['issues']
print("\n🐛 Issues Found:")
for issue in issues[:3]:
    print(f"  • {issue}")

suggestions = result.outputs['suggestions']
print("\n💡 Suggestions:")
for suggestion in suggestions[:3]:
    print(f"  • {suggestion}")

security_risks = result.outputs['security_risks']
print("\n🔒 Security Risks:")
for risk in security_risks[:3]:
    print(f"  ⚠️ {risk}")

# ChainOfThought adds reasoning
print(f"\n💭 Analysis reasoning: {result.outputs['reasoning'][:200]}...")

## ⚡ Performance Optimization Tips

<div style="background: #fff8e1; padding: 20px; border-radius: 10px; margin: 20px 0;">
    <h3 style="margin-top: 0;">🚀 Module Performance Guidelines</h3>
    <ul>
        <li><strong>Predict is fastest</strong> - Use for simple tasks</li>
        <li><strong>ChainOfThought adds 20-50% latency</strong> - Worth it for complex reasoning</li>
        <li><strong>Retry adds latency on failure</strong> - Configure delays wisely</li>
        <li><strong>Refine doubles+ the API calls</strong> - Use when quality is critical</li>
        <li><strong>Compose carefully</strong> - Each layer adds overhead</li>
    </ul>
</div>

In [ ]:
# Performance comparison
import asyncio

sig = "text -> summary, sentiment, keywords: list[str]"
test_text = "LogiLLM is an amazing framework for building LLM applications. It's fast and easy to use."

modules = {
    "Predict": Predict(sig),
    "ChainOfThought": ChainOfThought(sig),
    "Retry(Predict)": Retry(Predict(sig), max_retries=2),  # CORRECT PARAMETER
    "Retry(ChainOfThought)": Retry(ChainOfThought(sig), max_retries=2)  # CORRECT PARAMETER
}

print("⚡ Module Performance Comparison:\n")
print(f"{'Module':<25} {'Time (s)':<10} {'Relative'}")
print("-" * 45)

base_time = None
for name, module in modules.items():
    start = time.time()
    result = module.call_sync(text=test_text)  # Use call_sync() instead of await
    elapsed = time.time() - start
    
    if base_time is None:
        base_time = elapsed
        relative = "1.0x (baseline)"
    else:
        relative = f"{elapsed/base_time:.1f}x"
    
    print(f"{name:<25} {elapsed:<10.2f} {relative}")

print("\n💡 Choose modules based on your speed vs quality tradeoff!")

## 🏗️ Building Production Pipelines

Here's a real-world example combining everything:

In [ ]:
# Production-ready document processing pipeline
from logillm.core.adapters import JSONAdapter

class DocumentProcessor(Signature):
    """Process and analyze documents."""
    document: str = InputField(desc="Document text")
    doc_type: str = InputField(desc="Type: email, report, article, etc.")
    
    summary: str = OutputField(desc="Executive summary")
    key_points: list[str] = OutputField(desc="Main points")
    action_items: list[str] = OutputField(desc="Action items if any")
    sentiment: str = OutputField(desc="Overall sentiment")
    priority: str = OutputField(desc="high, medium, or low priority")

# Build production pipeline
def create_production_pipeline():
    """Create a robust document processing pipeline."""
    
    # Base module with reasoning and JSON adapter for proper list parsing
    base = ChainOfThought(DocumentProcessor, adapter=JSONAdapter())
    
    # Add retry for reliability
    reliable = Retry(
        base,
        max_retries=3,
        base_delay=1.0,
        backoff_multiplier=2.0
    )
    
    # Optional: Add refinement for critical documents
    # refined = Refine(
    #     module=reliable,
    #     N=2,
    #     reward_fn=lambda i, p: 0.8 if p.success else 0.0
    # )
    
    return reliable

# Create and test pipeline
pipeline = create_production_pipeline()

test_doc = """
Subject: Q3 Product Launch Update

Team,

Great progress on the new features! The beta testing revealed some issues 
with performance that need immediate attention. Marketing wants to move 
the launch date to next month. We need to decide by Friday.

Action items:
- Fix performance issues (Engineering)
- Finalize launch date (Product)
- Update marketing materials (Marketing)

This is critical for our Q3 goals.

Best,
Sarah
"""

print("🏭 Production Pipeline Test (with JSONAdapter):\n")
result = pipeline.call_sync(
    document=test_doc,
    doc_type="email"
)

print(f"📝 Summary: {result.outputs['summary']}")

# With JSONAdapter, lists are properly parsed
key_points = result.outputs['key_points']
print(f"\n🎯 Key Points:")
for point in key_points:
    print(f"  • {point}")

action_items = result.outputs['action_items']
print(f"\n✅ Action Items:")
for item in action_items:
    print(f"  • {item}")

print(f"\n😊 Sentiment: {result.outputs['sentiment']}")
print(f"🚨 Priority: {result.outputs['priority'].upper()}")

## 🎯 Summary & Key Takeaways

<div style="background: #e8f5e9; padding: 25px; border-radius: 10px; margin: 20px 0;">
    <h3 style="margin-top: 0;">What You've Learned</h3>
    <div style="display: grid; grid-template-columns: 1fr 1fr; gap: 20px;">
        <div>
            <h4>✅ Core Modules</h4>
            <ul>
                <li><strong>Predict</strong> - Fast and simple</li>
                <li><strong>ChainOfThought</strong> - Adds reasoning</li>
                <li><strong>Retry</strong> - Handles failures</li>
                <li><strong>Refine</strong> - Iterative improvement</li>
                <li><strong>ReAct</strong> - Tool use (preview)</li>
            </ul>
        </div>
        <div>
            <h4>✅ Key Concepts</h4>
            <ul>
                <li>Modules are execution strategies</li>
                <li>Different modules for different needs</li>
                <li>Modules can be composed</li>
                <li>Performance vs quality tradeoffs</li>
                <li>Production patterns</li>
            </ul>
        </div>
    </div>
</div>

## 🏁 Progress Check

In [ ]:
# Progress tracker
completed = {
    "module_basics": True,
    "predict": True,
    "chain_of_thought": True,
    "retry": True,
    "refine": True,
    "composition": True,
    "react_preview": True,
    "exercise": True,
    "performance": True,
    "production": True
}

total = len(completed)
done = sum(completed.values())
percentage = (done / total) * 100

print(f"📊 Notebook 3 Progress: {done}/{total} sections ({percentage:.0f}%)")
print("\n" + "█" * int(percentage // 5) + "░" * (20 - int(percentage // 5)))

if percentage == 100:
    print("\n🎉 Outstanding! You've mastered LogiLLM Modules!")
    print("Ready for Notebook 4: Providers & Adapters")

<div style="display: flex; justify-content: space-between; margin-top: 40px; padding: 20px; background: #f5f5f5; border-radius: 10px;">
    <a href="02_signatures.ipynb" style="text-decoration: none; padding: 10px 20px; background: white; border-radius: 5px; border: 1px solid #ddd;">← Notebook 2</a>
    <div style="text-align: center;">
        <strong>Great work! You're now a Module expert! 🎓</strong>
    </div>
    <a href="04_providers_adapters.ipynb" style="text-decoration: none; padding: 10px 20px; background: #667eea; color: white; border-radius: 5px;">Continue to Notebook 4 →</a>
</div>